In [ ]:
import random
import requests
import csv
import os
import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib.parse import quote
import re
import unicodedata


In [ ]:
#test2 #allot easier proably just going to use this
#creates inflections for nouns and adjectives
#creates conjugations for verbs
def strip_accents_and_lowercase(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn').lower()


def generate_variations(word,language='ancient greek',show=False):
        # CREATES A FOLDER STRUCTURE THAT CONTAINS WORDS ORGANIZED BY PART OF SPEECH
        #DECLINES WORDS
        #test words
        #'Ῥωμαῖος' romans
        #'μισώ' hate
        #'εἰμί' are to be
        #word = word.lower()

        #https://en.wiktionary.org/wiki/Ῥωμαῖος
        url= requests.get(f'https://en.wiktionary.org/wiki/{quote(word)}')
        soup = bs(url.text, 'html.parser')

        #ADDED ADDITION TO TRY AND ONLY GET ANCIENT GREEK. IN TESTING
        #if(soup.find('span', {'class':'mw-headline'}).text.lower().strip() == 'greek'):
          #print("Greek detected")
          #for i_tag in soup.find_all('i', {'class':'polytonic mention'}):
            #a_tag = i_tag.find('a')
            #if a_tag:
              #href = a_tag.get('href')
              #if(href and 'Ancient_Greek' in href):
                #url = requests.get(f'https://en.wiktionary.org/wiki/{quote(href)}')


        #remove unwanted classes
        for tag in soup.find_all('span',{'class':'tr Latn'}):
                tag.decompose()
        for tag in soup.find_all('table',{'class':'audiotable'}):
                tag.decompose()
        for tag in soup.find_all('sup'):
                tag.decompose()

        #HEADERS ---------------------------
        #find all tables
        tables =soup.find_all('table')
        part_of_speech_h3 =[]
        h4 =[] # inflection/declension
        nav=[] #verb tense, noun first second third delcension
        gender =None

        #DATAFRAMES----------------------------------

        dataframes = pd.read_html(soup.prettify())
        for i,df in enumerate(dataframes):
                
                header = tables[i].find_previous('h2').text.replace('[edit]','').lower().strip()

                #GENDER
                gender = tables[i].find_previous(class_="gender")
                if(gender != None):
                        gender = gender.find_next("abbr").text
                elif (gender ==None):
                        gender = "n"


                #if(header =='greek'):
                        # if modern greek ignore
                        #header = ''
                #if(header =='ancient greek'):
                        #header = 'greek'
                if(header != language):
                        part_of_speech_h3.append('')
                        h4.append('')
                        nav.append('')
                        continue
                if(tables[i].find_previous('h3')):
                        part_of_speech_h3.append(tables[i].find_previous('h3').text
                        .replace('[edit]','')
                        .replace('\"','').lower().strip())
                else:
                        part_of_speech_h3.append('')
                if(tables[i].find_previous('h4')):
                        h4.append(tables[i].find_previous('h4').text
                        .replace('[edit]','')
                        .replace('\"','').lower().strip())
                else:
                        h4.append('')
                if(tables[i].find_previous('div',{'class':'NavHead'})):
                        nav.append(tables[i].find_previous('div',{'class':'NavHead'}).text
                        .replace('hide ▲','')
                        .replace('\"','').replace('/','-').replace(':','-').strip().lower())
                        #  : / mess up the file path so you need to remove them
                else:
                        nav.append('')
                ##display(df)
                ##CREATE A FOLDER
                root_dir = f'{header.replace(" ", "_")}_variations'
                os.makedirs(f'{root_dir}/{part_of_speech_h3[i]}/{word.lower()}/',exist_ok=True)
                print(f'{root_dir}/{part_of_speech_h3[i]}/{word}/')
                ##os.makedirs(nav[i],exist_ok=True)

                #print(h3[i])
                #print(h4[i])
                #print(nav[i])
                #print('WRITE TO FILE--------------------------------')

                df=dataframes[i]

             
 
                # for appending df2 at the end of df1
                df.loc[len(df)]= gender
                df.iat[-1,0] = "gender"
                #df =pd.concat([df, pd.DataFrame(["gender",gender])], ignore_index=True)

                #df.append(pd.DataFrame({"a":[gender]}), ignore_index=True)
                #display(df)
                df=df.dropna(how='all',axis=0)
                #display(df.iloc[-1,0])
                #if(df.iloc[-1,0].lower()=='notes'or df.iloc[-1,0].lower()=='notes:'):
                        #df=df.drop(df.index[-1])

                #df=df.drop(list(df.filter(regex = 'Unnamed')), axis = 1)


                df=df.replace({'➤':''}, regex=True)
                df=df.replace({'•':''}, regex=True)
                df=df.replace({'\[':'','\]':'','\{':'','\}':''}, regex=True)
                df=df.replace({'—':' '}, regex=True)
                #df=df.replace({'/':' ',r'\\':' '}, regex=True)
                df=df.replace(r"\([^)]*\)","",regex=True)



                # make lowercase
                df = df.applymap(lambda s: s.lower() if type(s) == str else s)

                #df = df.applymap(lambda s:strip_accents_and_lowercase(s) if type(s) == str else s)
                df = df.applymap(lambda s: s.split(',', 1)[0] if type(s) == str else s)
                df = df.applymap(lambda s: s.split('-', 1)[0] if type(s) == str else s)
                df = df.applymap(lambda s: s.split('+', 1)[0] if type(s) == str else s)
                df = df.applymap(lambda s: s.split('/', 1)[0] if type(s) == str else s)
                df = df.applymap(lambda s: s.split('\\', 1)[0] if type(s) == str else s)

                df= df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


                df = df.replace(r"\s\s+"," ",regex=True)
              
                # NAN ARE REPALCED BY SPACE CHARACTER
                df.to_csv(f'{root_dir}/{part_of_speech_h3[i]}/{word.lower()}/{nav[i]}_{h4[i]}.csv',index=False)

                #display(df)
                if(show):display(df)
                ## READ
                #print('READ FROM FILE ------------------------------------')
                #display(pd.read_csv(f'{root}/{h3[i]}/{word}/{nav[i]}_{h4[i]}.csv'))
        #----------------------------------
#generate_variations('απεχθάνομαι','greek',show=True)
#generate_variations('ambulo','latin',show=True)
#test words
#'Ῥωμαῖος' romans
#'μισώ' hate
#'εἰμί' is/are to be
#ελληνικά greeks
#odium hatred
#romanus
#odio hate
#απεχθάνομαι
# Here, this generates greek output. But there is a class="polytonic mention" tag
# Maybe we can scrape the tag and set the url to the href inside
#lacedaemon

# TO SHOW THE DATAFRAME PASS SHOW AS TRUE

In [ ]:
###### HELPER FUNCTIONS#######


def path_to_folder(path):
    # converts non folders into folders
    #aka just adds a slash to the end, if you dont give it a folder then it will probably produce an errror
    root = f"{path}"
    if root[-1] != "/":
        root+="/"
    return root


def get_each_file_in_folder(path):
    # this only takes in a folder path
    #RETURNS A FILE LIST
    filelist =[]
    root =path_to_folder(path)
    for subdir, dirs, files in os.walk(root):
        for file in files:
            f = os.path.join(subdir, file)
            filelist.append(f)
    return filelist



def get_each_csv_in_folder(path):
    # takes in Noun, Adjective,Verb
    #Returns All csv's in a folder
    filelist =[]
    root =path_to_folder(path)
    for subdir, dirs, files in os.walk(root):
        for file in files:
            f = os.path.join(subdir, file)
            if f.endswith('.csv'):
                filelist.append(f)
    return filelist

def get_each_df_in_folder(path):
    #Takes in a folder
    #Gets every dataframe in a folder
    dflist =[]
    root =path_to_folder(path)
    for f in get_each_csv_in_folder(path):
        df = pd.read_csv(f).fillna('')

        dflist.append(df)
    return dflist
'''
def get_greek_df(path):
    # takes in Noun, Adjective,Verb
    #print("test")
    dflist =[]
    root =to_folder(path)
    for f in get_each_csv_in(path):
        df = pd.read_csv(f)
        greekdf=GreekCsv(f,df)
        dflist.append(greekdf)
    return dflist
'''
# get greek words from dictionary
def get_words_from_dict(path):
    wordlist = []
    for df in get_each_df_in_folder(path):
        for i in df.index:
             wordlist.append(df.iloc[i,1])
    return wordlist

#reads all csv and intializes all words into variations
def init_variations(path):
    # takes in a folder path
    # goes though every word in a dictionary and generates the variations of that word
    for word in get_words_from_dict(path):
        #display(df.iloc[i,1])
        generate_variations(word)

def match_words_in_folder(searchpath,wordpath,file_contains="",verbose=True):
#takes in a search path and a word path it will read a word from the word path and then find all
#files in each folder that contains that word
#returns a list of file paths
    filelist = []
    searchpath= path_to_folder(searchpath)
    for word in get_words_from_dict(wordpath):
        if(os.path.exists(searchpath + word)):
            filelist.append(get_each_file_in_folder(searchpath + word))
    flat_list = [item for sublist in filelist for item in sublist]
    if(file_contains!=""):
        for path in flat_list:
            if ((file_contains in path)== False ):
                flat_list.remove(path)
    if verbose:
        print(flat_list)
    return flat_list
def get_df_from_csv(file):
    df =pd.read_csv(file,header=None).fillna('')
    df= df.applymap(lambda s:s.lower() if type(s) == str else s)
    return df
def get_each_df_from_path_list(filelist):
    #takes in a list of csv paths and outputs a list of dataframes
    dflist=[]
    for file in filelist:
        df = get_df_from_csv(file)
        dflist.append(df)
    return dflist

def display_list(list):
# takes in a list of dataframes and displays the data
    for i in list:
        display(i)

def display_all():
    init_variations("dictionary/")
    matchedwords= match_words_in_folder("variations/")
    display_list(get_each_df_from_path_list(matchedwords))




def get_indices(df,words):
    # returns a tuple of 2 lists that contain the row indexes and column inexes for each column and row that this word is present in
    # convert to take in a list of words
    columns=[]
    rows = []
    for columnIndex in range(df.shape[1]):
        if(isinstance(words, list)):
            for word in words:
                rowIndex = df.index[df.iloc[:, columnIndex].str.contains(word)].tolist()
                if(rowIndex):
                    columns.append(columnIndex)
                    rows.append(rowIndex)
        else:
            rowIndex = df.index[df.iloc[:, columnIndex].str.contains(words)].tolist()
            if(rowIndex):
                columns.append(columnIndex)
                rows.append(rowIndex)

            #remove duplicates
    if(len(rows)!=0):    
        rows = [item for sublist in rows for item in sublist]
        rows = [*set(rows)]
        
    if(len(rows)==0): 
        return ([''],[''])
    if(len(columns)==0):
        return ([''],[''])
    #print("COLUMNINDEXES ",columns)
    #print("ROWINDEXES ",rows)
    
    return (rows,columns)


def get_items(dflist,row_words,column_words=''):
    # takes in a tuple of index arrays
    # empty strings are used to indicate not input standard input is either a string or array of strings
    dflist2=[]
    if (row_words=='' and column_words==''):
            print('please put a word to search in columns or rows')
            return []

    for df in dflist:#
        row_indexes = get_indices(df,row_words)[0]
        column_indexes = get_indices(df,column_words)[1]
        if (row_words=='' and column_words!=''):
            df2 =df.iloc[:,column_indexes]
        if (row_words!='' and  column_words==''):
            df2 =df.iloc[row_indexes]

        df2 =df.iloc[row_indexes,column_indexes]

        dflist2.append(df2)
    return dflist2

# a function that checks if a string contains a number
def contains_number(string):
    return any(char.isdigit() for char in string)
